In [63]:
import os
import pandas as pd
from sqlalchemy import create_engine

SERVER_BD = "localhost\\SQLEXPRESS"
DATABASE = "AdventureWorks2022"

## 2.1. Agregación de columnas y filtrado

### <b>1. Seleccionar todos los dinosaurios en un solo DataFrame</b>

In [64]:
#conectar a Dinosaurs.csv
df_dinos = pd.read_csv("C:\\Users\\miguel.fuentes.vale\\Documents\\Apuntes_MongoDB\\dinosaur.csv", delimiter=",")

print(df_dinos.head(10))

            Name             Period       Diet        Country
0       Aardonyx           Jurassic  herbivore   South Africa
1    Abelisaurus  Cretaceous         carnivore  South America
2  Abrictosaurus           Jurassic  herbivore   South Africa
3     Abrosaurus           Jurassic  herbivore          China
4    Abydosaurus         Cretaceous  herbivore  North America
5  Acanthopholis         Cretaceous  herbivore        England
6  Achelousaurus         Cretaceous  herbivore  North America
7   Acheroraptor         Cretaceous  carnivore  North America
8  Achillesaurus         Cretaceous  carnivore  South America
9   Achillobator         Cretaceous  carnivore       Mongolia


### <b>2. Agregar una columna con IDs únicos para cada dinosaurio en la lista</b>

In [65]:
df_dinos["ID"] = df_dinos["Name"].index

print(df_dinos["ID"].dtype) # comprobar que el tipo de dato de la columna ID es int
print(df_dinos)

int64
                Name             Period       Diet        Country    ID
0           Aardonyx           Jurassic  herbivore   South Africa     0
1        Abelisaurus  Cretaceous         carnivore  South America     1
2      Abrictosaurus           Jurassic  herbivore   South Africa     2
3         Abrosaurus           Jurassic  herbivore          China     3
4        Abydosaurus         Cretaceous  herbivore  North America     4
...              ...                ...        ...            ...   ...
1149  Zizhongosaurus           Jurassic  herbivore          China  1149
1150    Zuniceratops         Cretaceous  herbivore  North America  1150
1151         Zuolong           Jurassic  carnivore          China  1151
1152     Zupaysaurus           Triassic  carnivore  South America  1152
1153            Zuul         Cretaceous  herbivore  North America  1153

[1154 rows x 5 columns]


### <b>2.1. exportar a SQL Server (nombre de tabla libre)</b>

In [71]:
# Hacer la conexión, y guardar la nueva tabla
conn = create_engine(f"mssql+pyodbc:///?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER={SERVER_BD};DATABASE={DATABASE};Trusted_Connection=yes")
df_dinos.to_sql("DINOSAURIOS_py", conn)

107

### <b>3. A partir del DataFrame resultante, mostrar por pantalla sólo aquellos dinosaurios con ID múltiplo de 10</b>

In [73]:
df_mult_10 = df_dinos.query("ID % 10 == 0")
print(df_mult_10.head(20))

              Name           Period         Diet        Country   ID
0         Aardonyx         Jurassic    herbivore   South Africa    0
10      Acristavus       Cretaceous    herbivore  North America   10
20      Aepisaurus       Cretaceous    herbivore         France   20
30    Airakoraptor       Cretaceous    carnivore       Mongolia   30
40       Albinykus       Cretaceous    carnivore       Mongolia   40
50        Alocodon         Jurassic  (herbivore)       Portugal   50
60    Amphicoelias         Jurassic    herbivore  North America   60
70      Andesaurus       Cretaceous    herbivore  South America   70
80   Antarctopelta  Late Cretaceous    herbivore     Antarctica   80
90    Aquilarhinus       Cretaceous    herbivore  North America   90
100    Arenysaurus       Cretaceous    herbivore          Spain  100
110       Astrodon       Cretaceous    herbivore  North America  110
120     Aucasaurus       Cretaceous    carnivore  South America  120
130   Aviatyrannis         Jurassi

### <b>4. Ordenar los resultados de manera ascendente por periodo (Period)</b>

In [99]:
df_mult_10 = df_mult_10.sort_values("Period", ascending=True)
print(df_mult_10.head(20))

                  Name      Period                Diet  \
1150      Zuniceratops  Cretaceous           herbivore   
850       Quilmesaurus  Cretaceous           carnivore   
1140     Zephyrosaurus  Cretaceous           herbivore   
400       Futabasaurus  Cretaceous           carnivore   
540      Kelmayisaurus  Cretaceous           carnivore   
880       Sanchusaurus  Cretaceous           herbivore   
520      Jiangxisaurus  Cretaceous            omnivore   
510       Iuticosaurus  Cretaceous           herbivore   
840    Pycnonemosaurus  Cretaceous           carnivore   
860        Regnosaurus  Cretaceous           herbivore   
910          Sellacoxa  Cretaceous           herbivore   
930   Sigilmassasaurus  Cretaceous           carnivore   
460               Haya  Cretaceous  herbivore/omnivore   
940   Sinornithosaurus  Cretaceous           carnivore   
440        Goyocephale  Cretaceous  herbivore/omnivore   
430         Gobiraptor  Cretaceous          omnivorous   
420      Gigan

## 2.2. Agrupación y funciones de agregación

### <b>1. Encontrar la cantidad de dinosaurios por tipo de alimentación (Diet)</b>

In [159]:
#En SQL, se haría groupby y COUNT sobre la columna ID/Name(la que tenga valores únicos)
df_dinos_group = df_dinos[["ID", "Diet"]].groupby("Diet", as_index=False).agg( # Usamos "as_index=False" para poder seleccionar la columna aggregada más tarde
    n_dinos=pd.NamedAgg("ID", aggfunc="count")
)

print(df_dinos_group.head(10))

                    Diet  n_dinos
0            (carnivore)        3
1            (herbivore)        1
2              (unknown)        7
3                      ?        2
4              Carnivore        4
5              Herbivore        2
6              carnivore      340
7  carnivore/insectivore        1
8     carnivore/omnivore        4
9             carnivore?        3


### <b>2. A partir de este resultado, elimina los paréntesis y coloca la primera letra de cada dieta (Diet) en mayúscula</b>

In [162]:
# Realizamos una copia para evitar perder la info del DF original
df_dinos_group2 = df_dinos_group.copy(deep=True)

# Eliminar los paréntesis usando el método apply() de pandas ;; Se puede hacer mayúscula la primera letra con capitalize()
df_dinos_group2["Diet"] = df_dinos_group2["Diet"].apply(lambda x: x[1:-1] if (x.startswith("(")) else x)
df_dinos_group2["Diet"] = df_dinos_group2["Diet"].str.capitalize()
# Se vuelve a agrupar, si usáramos drop_duplicates() perderíamos la info. de "n_dinos"
df_dinos_group2 = df_dinos_group2.groupby("Diet", as_index=False).agg(
    n_dinos=pd.NamedAgg("n_dinos", aggfunc="count")
)

print(df_dinos_group2)

                     Diet  n_dinos
0                       ?        1
1               Carnivore        3
2   Carnivore/insectivore        1
3      Carnivore/omnivore        1
4              Carnivore?        1
5               Herbivore        3
6      Herbivore/omnivore        1
7             Insectivore        1
8                Omnivore        1
9              Omnivorous        1
10                Unknown        2


### <b>3.Ordenar los resultados por tipo de alimentación</b>

In [165]:
df_dinos_group2 = df_dinos_group2.sort_values("Diet")
print(df_dinos_group2)

                     Diet  n_dinos
0                       ?        1
1               Carnivore        3
2   Carnivore/insectivore        1
3      Carnivore/omnivore        1
4              Carnivore?        1
5               Herbivore        3
6      Herbivore/omnivore        1
7             Insectivore        1
8                Omnivore        1
9              Omnivorous        1
10                Unknown        2


### <b>4- Hacer el equivalente a un UNION (sin duplicados) con el dataset(CSV) ó tabla original</b>

In [172]:
#Primero, vamos a obtener ambos dataframes a los que hacer UNION
df1 = df_dinos #el original, con IDs
df2 = df_dinos_group2

# Ahora, usaremos el método CONCAT de pandas para realizar el equivalente a UNION en SQL
df1_df2_union = pd.concat([df1, df2], ignore_index=True).drop_duplicates()
print(df1_df2_union.tail(40))

                  Name      Period                   Diet  \
1125      Yizhousaurus    Jurassic              herbivore   
1126      Yongjinglong  Cretaceous              herbivore   
1127     Yuanmousaurus    Jurassic              herbivore   
1128        Yueosaurus  Cretaceous              herbivore   
1129            Yulong  Cretaceous               omnivore   
1130       Yunganglong  Cretaceous              herbivore   
1131     Yunnanosaurus    Jurassic              herbivore   
1132    Yunxianosaurus  Cretaceous              herbivore   
1133       Yurgovuchia  Cretaceous              carnivore   
1134        Yutyrannus  Cretaceous              carnivore   
1135          Zalmoxes  Cretaceous              herbivore   
1136         Zanabazar  Cretaceous              carnivore   
1137      Zapalasaurus  Cretaceous              herbivore   
1138          Zapsalis  Cretaceous              carnivore   
1139               Zby    Jurassic              herbivore   
1140     Zephyrosaurus  

### <b>5- Si hacemos un tail() al DF original (o al resultante), nos daremos cuenta de que hay algunos datos en la columna Country que no tienen que ver con el país/origen de los dinosaurios. Pasa aquellos valores que cumplan estas condiciones a una nueva columna llamada Descripcion.</b>
PISTA: usa la función de python split() para lograrlo


In [202]:
print("Tail 30:")
print(df_dinos.tail(10))

#Hacer una copia, y pasar los valores de "Country" que sean descripciones a una columna "Descripcion"
df_descrips = df_dinos.copy(deep=True)
df_descrips["Descripcion"] = df_descrips["Country"].apply(lambda x: x if (len(x.split(" ")) >= 3) else "Dinosaurio")

# Quitar los valores de Country que tengan una "descripcion"
df_descrips["Country"] = df_descrips["Country"].apply(lambda x: x if (len(x.split(" ")) <= 2) else "Origen desconocido")

print("Resultado:")
print(df_descrips.tail(30))

Tail 30:
                  Name      Period       Diet        Country    ID
1144   Zhongyuansaurus  Cretaceous  herbivore          China  1144
1145  Zhuchengceratops  Cretaceous  herbivore          China  1145
1146  Zhuchengtyrannus  Cretaceous  carnivore          China  1146
1147          Ziapelta  Cretaceous  herbivore  North America  1147
1148     Zigongosaurus    Jurassic  herbivore          China  1148
1149    Zizhongosaurus    Jurassic  herbivore          China  1149
1150      Zuniceratops  Cretaceous  herbivore  North America  1150
1151           Zuolong    Jurassic  carnivore          China  1151
1152       Zupaysaurus    Triassic  carnivore  South America  1152
1153              Zuul  Cretaceous  herbivore  North America  1153
Resultado:
                  Name      Period       Diet             Country    ID  \
1124     Yixianosaurus  Cretaceous  carnivore  Origen desconocido  1124   
1125      Yizhousaurus    Jurassic  herbivore  Origen desconocido  1125   
1126      Yongjing